In [43]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException


import pandas as pd
import numpy as np

https://www.airnav.com/airport/KXWA
https://adip.faa.gov/agis/public/#/airportSearch

In [69]:
flights_fe = pd.read_csv(r"../data/sample_with_inbound_data.csv")

In [70]:
airport_codes = flights_fe.origin.replace({'FCA':'GPI','ISN':'XWA','SCE':'UNV','BKG':'BBG','AZA':'IWA','USA':'JQF',
                      'MQT':'SAW','YUM':'NYL','SPN':'GSN','HHH':'HXD'}).unique()

['ADK', 'FCA', 'ISN', 'SCE', 'JHM', 'BKG', 'AZA', 'USA', 'MQT',
       'YUM', 'SPN', 'HHH']

['PADK','GPI','XWA','UNV','PHJH','BBG','IWA','JQF','SAW',
'NYL','GSN','HXD']

In [71]:

# airports_not_avilable_in_faa = ['FCA', 'ISN', 'SCE', 'BKG', 'AZA', 'USA', 'MQT', 'YUM', 'SPN', 'HHH']

# new_airport_codes = set(airport_codes).difference(airports_not_avilable_in_faa)

In [72]:
css_selectors = ["body > main:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > airport-details:nth-child(1) > div:nth-child(4) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2)",
                 "div[ng-if='airportDetailData.npiasNumber'] div[class='col-md-8 ng-binding']",
                "div[ng-if='airportDetailData.serviceLevel'] div[class='col-md-8 ng-binding']",
                "div[ng-if='airportDetailData.hubType'] div[class='col-md-8 ng-binding']",
                "div[ng-if='airportDetailData.status'] div[class='col-md-8 ng-binding']",
                "span[ng-bind='ddToDms(airportDetailData.arpLatitude, true, false)']",
                 "span[ng-bind='ddToDms(airportDetailData.arpLongitude, true, true)']",
                "body > main:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > airport-details:nth-child(1) > div:nth-child(4) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(7) > div:nth-child(2)",
                "div[class='card mb-2 ng-scope'] div[class='row'] div:nth-child(1) div:nth-child(8) div:nth-child(2)",
                "p[class='m-0 ng-binding']",
                 "div[ng-if='airportDetailData.magneticVariation'] div[class='col-md-8 ng-binding']",
                 "div[class='card-body py-3'] div:nth-child(1) div:nth-child(11) div:nth-child(2)",
                "div[ng-if='airportDetailData.ownershipType'] div[class='col-md-8 ng-binding']",
                "div[ng-if='airportDetailData.facilityUse'] div[class='col-md-8 ng-binding']",
                 "div[ng-if='airportDetailData.areaInAcres'] div[class='col-md-8 ng-binding']",
                "div[ng-if='airportDetailData.tieInFssFlag'] div[class='col-md-8 ng-binding']",
                "div[ng-if='airportDetailData.beaconLensColor'] div[class='col-md-8 ng-binding']",
                "body > main:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > airport-details:nth-child(1) > div:nth-child(4) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(2)",
                "body > main:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > airport-details:nth-child(1) > div:nth-child(4) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(2) > div:nth-child(6) > div:nth-child(2)",
                "body > main:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > airport-details:nth-child(1) > div:nth-child(4) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(2) > div:nth-child(7) > div:nth-child(2)"]

In [73]:
variables = ["site_id","npias_no","service_level","hub_type","airport_status","lat","lon","city","county","elevation",
             "variation","last_inspection_date",
             "ownership_status","facility_use","area","fss_on_airport","beacon_color","air_carrier_annual",
            "total_operations_annual","operations_annual_ending","num_runways","avg_runway_length","avg_runway_width"]

In [74]:
#This example requires Selenium WebDriver 3.13 or newer
with webdriver.Chrome() as driver:
    new_airport_details = []
    for airport_code in airport_codes:
        url = f"https://adip.faa.gov/agis/public/#/airportData/{airport_code}"
        driver.implicitly_wait(10) # seconds
        driver.get(url)
        results = []
        for selector in css_selectors:
            try:
                result = driver.find_element_by_css_selector(selector)
                results.append(result.text)
            except NoSuchElementException as exc:
                print(f"{airport_code} : {exc}")
                result = ""
                results.append(result)
        we = driver.find_elements_by_css_selector("div[title='Dimensions, surfaces, edge lights, weight limits']")
        length=[]
        width = []
        num_runways = len(we)
        for runway in we:
            try:
                runway_dims = runway.find_element_by_xpath(".//div[contains(text(),'ft.')]").text
                dims = runway_dims.split("x")
                length.append(float(dims[0].rstrip(" ft. ")))
                width.append(float(dims[1].rstrip(" ft. ")))
            except NoSuchElementException:
                num_runways = num_runways-1
        results.append(num_runways)
        results.append(np.mean(length))
        results.append(np.mean(width))
        details = dict(zip(variables,results))
        details["airport_code"] = airport_code
        new_airport_details.append(details)

ADQ : Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[ng-if='airportDetailData.areaInAcres'] div[class='col-md-8 ng-binding']"}
  (Session info: chrome=92.0.4515.107)

ADK : Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[ng-if='airportDetailData.areaInAcres'] div[class='col-md-8 ng-binding']"}
  (Session info: chrome=92.0.4515.107)

OME : Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[ng-if='airportDetailData.areaInAcres'] div[class='col-md-8 ng-binding']"}
  (Session info: chrome=92.0.4515.107)

BRW : Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[ng-if='airportDetailData.areaInAcres'] div[class='col-md-8 ng-binding']"}
  (Session info: chrome=92.0.4515.107)

YAK : Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[ng-if='airportDetailData.areaInAcres'] div[class=

In [75]:
data = pd.DataFrame(new_airport_details)

In [76]:
data.head()

,site_id,npias_no,service_level,hub_type,airport_status,lat,lon,city,county,elevation,...,area,fss_on_airport,beacon_color,air_carrier_annual,total_operations_annual,operations_annual_ending,num_runways,avg_runway_length,avg_runway_width,airport_code
0,03640.*A,13-0008,Primary,Large,Operational,"33° 38' 12.1186"" N","84° 25' 40.3104"" W","ATLANTA, GA",FULTON,1026.2 ft.,...,4700 ac.,NO,Clear and Green,"474,822","525,305",01/31/2021,5,9877.800000,150.000000,ATL
1,09749.*A,26-0026,Primary,Large,Operational,"42° 12' 44.8"" N","83° 21' 12.2"" W","DETROIT, MI",WAYNE,645.2 ft.,...,4850 ac.,NO,Clear and Green,"315,995","393,681",12/31/2018,6,9618.833333,158.333333,DTW
2,27013.1*A,55-0002,Primary,Non-Hub,Operational,"44° 15' 29.1"" N","88° 31' 8.7"" W","APPLETON, WI",OUTAGAMIE,918.3 ft.,...,1638 ac.,NO,Clear and Green,"6,490","37,809",12/31/2019,2,7251.500000,150.000000,ATW
3,03728.*A,13-0035,Primary,Non-Hub,Operational,"32° 30' 58.8"" N","84° 56' 19.9"" W","COLUMBUS, GA",MUSCOGEE,397.4 ft.,...,680 ac.,NO,Clear and Green,348,"36,760",12/31/2019,2,5497.000000,112.500000,CSG
4,07425.*A,22-0006,Primary,Non-Hub,Operational,"30° 31' 58.5"" N","91° 8' 59.6"" W","BATON ROUGE, LA",EAST BATON ROUGE,69.7 ft.,...,1250 ac.,NO,Clear and Green,"11,908","55,331",08/31/2018,3,6101.333333,125.000000,BTR


In [77]:
data.elevation = data.elevation.map(lambda x: x.rstrip("ft. "))
data.area = data.area.map(lambda x: x.rstrip("ac."))
data.air_carrier_annual = data.air_carrier_annual.map(lambda x: x.replace(',',''))
data.total_operations_annual = data.total_operations_annual.map(lambda x: x.replace(',',''))

In [78]:
data.head()

,site_id,npias_no,service_level,hub_type,airport_status,lat,lon,city,county,elevation,...,area,fss_on_airport,beacon_color,air_carrier_annual,total_operations_annual,operations_annual_ending,num_runways,avg_runway_length,avg_runway_width,airport_code
0,03640.*A,13-0008,Primary,Large,Operational,"33° 38' 12.1186"" N","84° 25' 40.3104"" W","ATLANTA, GA",FULTON,1026.2,...,4700,NO,Clear and Green,474822,525305,01/31/2021,5,9877.800000,150.000000,ATL
1,09749.*A,26-0026,Primary,Large,Operational,"42° 12' 44.8"" N","83° 21' 12.2"" W","DETROIT, MI",WAYNE,645.2,...,4850,NO,Clear and Green,315995,393681,12/31/2018,6,9618.833333,158.333333,DTW
2,27013.1*A,55-0002,Primary,Non-Hub,Operational,"44° 15' 29.1"" N","88° 31' 8.7"" W","APPLETON, WI",OUTAGAMIE,918.3,...,1638,NO,Clear and Green,6490,37809,12/31/2019,2,7251.500000,150.000000,ATW
3,03728.*A,13-0035,Primary,Non-Hub,Operational,"32° 30' 58.8"" N","84° 56' 19.9"" W","COLUMBUS, GA",MUSCOGEE,397.4,...,680,NO,Clear and Green,348,36760,12/31/2019,2,5497.000000,112.500000,CSG
4,07425.*A,22-0006,Primary,Non-Hub,Operational,"30° 31' 58.5"" N","91° 8' 59.6"" W","BATON ROUGE, LA",EAST BATON ROUGE,69.7,...,1250,NO,Clear and Green,11908,55331,08/31/2018,3,6101.333333,125.000000,BTR


In [79]:
enplanements = pd.read_excel("../data/preliminary-cy20-commercial-service-enplanements.xlsx")

In [80]:
enplanements.head()

,Rank,FAA Region,ST,Locid,City,Airport Name,S/L,Hub,CY 20 Enplanements,CY 19 Enplanements,% Change
0,1,SO,GA,ATL,Atlanta,Hartsfield - Jackson Atlanta International,P,L,20559866.0,53505795.0,-0.6157
1,2,SW,TX,DFW,Fort Worth,Dallas-Fort Worth International,P,L,18593421.0,35778573.0,-0.4803
2,3,NM,CO,DEN,Denver,Denver International,P,L,16243349.0,33592945.0,-0.5165
3,4,GL,IL,ORD,Chicago,Chicago O'Hare International,P,L,14606034.0,40871223.0,-0.6426
4,5,WP,CA,LAX,Los Angeles,Los Angeles International,P,L,14059754.0,42939104.0,-0.6726


In [81]:
test_enplanements = enplanements[["Rank","CY 19 Enplanements","Locid"]]

In [82]:
test_enplanements.Locid = test_enplanements.Locid.replace({'FCA':'GPI','ISN':'XWA','SCE':'UNV','BKG':'BBG','AZA':'IWA','USA':'JQF',
                      'MQT':'SAW','YUM':'NYL','SPN':'GSN','HHH':'HXD'})

D:\Anaconda\envs\geo_env\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [83]:
data = pd.merge(left=data,right=test_enplanements,how="left",left_on="airport_code",right_on="Locid")

In [84]:
data.to_csv(r"../data/faa_airport_details.csv",index=False)

In [ ]:
//h3[normalize-space()='Runway Information']

In [60]:
with webdriver.Chrome() as driver:
    airport_code = "JFK"
    url = f"https://adip.faa.gov/agis/public/#/airportData/{airport_code}"
    driver.implicitly_wait(10) # seconds
    driver.get(url)


Runway - 04L/22R
12079 ft. x 200 ft.
Runway - 04R/22L
8400 ft. x 200 ft.
Runway - 13L/31R
10000 ft. x 200 ft.
Runway - 13R/31L
14511 ft. x 200 ft.


In [65]:
dims = "12079 ft. x 200 ft."
for dim in dims:
    print(dim.rstrip(" ft. "))

12079
 200
